In [53]:
!pip install python-docx 

   ---------------------------------------- 0.0/244.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/244.3 kB ? eta -:--:--
   --- ----------------------------------- 20.5/244.3 kB 217.9 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/244.3 kB 281.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/244.3 kB 525.1 kB/s eta 0:00:01
   ----------------------------------- -- 225.3/244.3 kB 981.9 kB/s eta 0:00:01
   ---------------------------------------- 244.3/244.3 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\serzh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import requests

In [54]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
from docx import Document

In [27]:
def load_and_split_documents(documents_folder):
    """
    Загрузка документов из локальной директории и разбиение их на фрагменты.
    """
    # Загрузка документов с указанием кодировки utf-8
    loader = DirectoryLoader(
        documents_folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
    )
    documents = loader.load()
    
    # Разбиение документов на фрагменты
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    return texts

In [56]:
def convert_docx_to_txt(folder_path):
    # Проверяем, существует ли папка
    if not os.path.isdir(folder_path):
        print(f"Папка {folder_path} не найдена.")
        return

    # Проходим по всем файлам в указанной папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):  # Проверяем, что это .docx файл
            docx_path = os.path.join(folder_path, filename)
            txt_path = os.path.join(folder_path, filename.replace('.docx', '.txt'))

            # Чтение содержимого .docx файла
            doc = Document(docx_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"

            # Запись содержимого в .txt файл
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)

            print(f"Файл {filename} успешно преобразован в {txt_path}")

# Пример использования
documents_folder = "..\documents"  # Путь к папке с документами
convert_docx_to_txt(documents_folder)


Файл Книга 1 - Инвентаризация карьер Большегорский.docx успешно преобразован в ..\documents\Книга 1 - Инвентаризация карьер Большегорский.txt
Файл Книга 2 - ПДВ карьер Большегорский.docx успешно преобразован в ..\documents\Книга 2 - ПДВ карьер Большегорский.txt


In [40]:
# Оптимизированная функция для создания эмбеддингов с mean pooling
def create_embeddings(texts, tokenizer, model):
    # Токенизация и создание эмбеддингов
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling с учетом маски внимания
    attention_mask = inputs['attention_mask']
    token_embeddings = outputs[0]
    mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    embeddings = torch.sum(token_embeddings * mask_expanded, dim=1) / torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
    
    return embeddings

In [47]:
# Оптимизированная функция для создания векторного хранилища ChromaDB
def create_vectorstore(texts, model_name="ai-forever/sbert_large_nlu_ru"):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Загрузка токенизатора и модели только один раз
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    print('модели скачаны')
    # Преобразование документов и создание эмбеддингов
    text_data = [doc.page_content for doc in texts]
    embeddings = create_embeddings(text_data, tokenizer, model)  # Передаем все тексты сразу для повышения эффективности

    # Создание и сохранение векторного хранилища ChromaDB
    vectorstore = Chroma.from_embeddings(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    print('данные преобразованы')
    vectorstore.persist()
    return vectorstore


In [30]:
def call_api(prompt: str) -> str:
    url = "YOUR_API_ENDPOINT"  # Замените на фактический URL API
    headers = {
        "Content-Type": "application/json",
        # "Authorization": "Bearer YOUR_API_KEY"  # Если требуется аутентификация
    }
    payload = {
        "modelUri": "gpt://<идентификатор_каталога>/yandexgpt-lite",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": "2000"
        },
        "messages": [
            {
                "role": "system",
                "text": "Найди ошибки в тексте и исправь их"
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        # Извлечение сгенерированного текста из ответа
        generated_text = result.get('text', '')
        return generated_text
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

In [31]:
def run_query(retriever, query):
    """
    Запуск поиска и генерация ответа с использованием API.
    """
    # Получение релевантных документов
    docs = retriever.get_relevant_documents(query)
    
    # Формирование контекста из документов
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Создание полного запроса с учетом контекста
    prompt = f"Контекст:\n{context}\n\nВопрос:\n{query}"
    
    # Вызов API с предоставленным промптом
    response = call_api(prompt)
    
    # Возврат результата и исходных документов
    result = {
        "result": response,
        "source_documents": docs
    }
    return result

In [42]:
documents_folder = "..\documents"  # Путь к папке с документами
query = "Какие существуют типы источников выбросов? Привести примеры каждого источника."  # Пользовательский запрос

In [43]:
# Загрузка и разбиение документов
texts = load_and_split_documents(documents_folder)

In [44]:
texts = texts[:10]  # Ограничение количества документов
texts, len(texts)

([Document(page_content='А.С. СТЕПАНОВСКИХ\n\n\n\n\nЭКОЛОГИЯ\n\n\n\n\n\nРекомендовано Министерством образования \nРоссийской Федерации в качестве учебника \nдля студентов высших учебных заведений\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nМосква» 2001\n\n\n\n\nУДК 574(075.8) \nББК 28.081 \nС 79\n\nРекомендовано Учебно-методическим центром «Профессиональный учебник» в качестве учебника для студентов высших учебных заведений\n\nРецензенты:\nд-р с.-х. наук, проф.. Засл. деятель науки РФ А.Г. Таскаева\n (Челябинский агроинженерный университет);\nд-р биол. наук, проф. T.В. Теплякова \n(Сибирский университет потребительской кооперации);\nд-р с.-х. наук, проф., Засл. деятель науки РФ В.А. Чулкина \n(Новосибирский аграрный университет)\n\nГлавный редактор издательства Н.Д. Эриашвили\n\nСтепановских А.С.\nС 79   Экология: Учебник для вузов. — М.: ЮНИТИ-ДАНА, 2001. - 703 с. \n\n\nISBN 5-238-00284-Х', metadata={'source': '..\\documents\\экология учебник.txt'}),
  Document(page_content='ISBN 5-238-00284-Х\n\n\n

In [45]:
len(texts[0].page_content)

809

In [51]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

KeyboardInterrupt: 

In [48]:
# Создание векторного хранилища
vectorstore = create_vectorstore(texts)

C:\Users\serzh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\serzh\.cache\huggingface\hub\models--ai-forever--sbert_large_nlu_ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downlo

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.hf.co\', port=443): Max retries exceeded with url: /sberbank-ai/sbert_large_nlu_ru/cea5e5ebffd98391d7c119f2d35a50e546aad6aea7c883bb584754874d27f622?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1730152733&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDE1MjczM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zYmVyYmFuay1haS9zYmVydF9sYXJnZV9ubHVfcnUvY2VhNWU1ZWJmZmQ5ODM5MWQ3YzExOWYyZDM1YTUwZTU0NmFhZDZhZWE3Yzg4M2JiNTg0NzU0ODc0ZDI3ZjYyMj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=mFdPT7zMESfBzADjIFc5lBHRNwFKfUNk~PyKoo1Blh5LzLYdPbIFUGa05Tj9GEC5hp29X0QJOurQ6phScesdKKEPiD22naognOqbkXoBS5QyZ~w5rMcYH7FKG~XNLAntW8qYWw56zAi3Lnl5yn1GaE~odW0qgTYVqV4OG07PvERxdZZ-6YH-Lzyf4rbp2qBfVnqtQ0WmseusIVYyd6YFePQjT3l8EftxP7oIsN-e-HaQK3prOz~0n7xM8IVT-az~D8zml9t8PBjvIbd5DANq9IS5QqvKZRFZLK1ZuJVEAdu4Vlpsgys2qL

In [ ]:
vectorstore

In [ ]:






# Настройка ретривера
retriever = vectorstore.as_retriever()

# Запуск запроса
result = run_query(retriever, query)

# Вывод ответа и исходных документов
print("Ответ:")
print(result["result"])
print("\nИсходные документы:")
for doc in result["source_documents"]:
print(doc.metadata["source"])

In [ ]:
# Основное выполнение
if __name__ == "__main__":
    # Параметры
    documents_folder = "./documents"  # Путь к папке с документами
    query = "В чем смысл жизни?"  # Пользовательский запрос
    
    # Загрузка и разбиение документов
    texts = load_and_split_documents(documents_folder)
    
    # Создание векторного хранилища
    vectorstore = create_vectorstore(texts)
    
    # Настройка ретривера
    retriever = vectorstore.as_retriever()
    
    # Запуск запроса
    result = run_query(retriever, query)
    
    # Вывод ответа и исходных документов
    print("Ответ:")
    print(result["result"])
    print("\nИсходные документы:")
    for doc in result["source_documents"]:
        print(doc.metadata["source"])


In [4]:
def create_vectorstore(texts, embedding_model_name):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Создание эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    
    # Создание векторного хранилища ChromaDB
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    vectorstore.persist()
    return vectorstore